In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import datetime
from datetime import time,datetime

def gammaCorrection(frame, gamma):
    invGamma = 1 / gamma
    table = [((i / 255) ** invGamma) * 255 for i in range(256)]
    table = np.array(table, np.uint8)
    return cv2.LUT(frame, table)

def get_binary_image(frame):
    enhanced_frame = gammaCorrection(frame, 2)
    gray_frame = cv2.GaussianBlur(cv2.cvtColor(enhanced_frame, cv2.COLOR_RGB2GRAY),
                                  (5,5),0)
    threshold = cv2.threshold(gray_frame, 200,255, cv2.THRESH_BINARY)[1]
    return (enhanced_frame, threshold)

def display_tracking_window_axis_center(frame,x,y,tw_width,tw_height, rows):
        bounding_box = cv2.rectangle(frame,
                  (x, y),
                  (x + tw_width, y + tw_height),
                  (255, 0, 0), 2)
        horisontal_axis = cv2.line(frame,
                        (x + int(tw_width/2), 0),
                        (x + int(tw_width/2), rows),
                        (255, 0, 0), 2)
        object_center = cv2.circle(frame,
                                (x + int(tw_width/2), y + int(tw_height/2)),
                                2, (255,0,0),2)
        
        return (bounding_box, horisontal_axis,object_center)

def draw_lines(frame, l, y, tw_height):
     line = [((l-1), y), ((l-1), tw_height),((l+1), tw_height),((l+1), y)]
     cv2.polylines(frame,[np.array(line, np.int32)],True, (0,0,255), 1)


def check_line_crossing(l,x,y,tw_width,tw_height):
    line = [((l-1), y), ((l-1), tw_height),((l+1), tw_height),((l+1), y)]
    line_crossing = cv2.pointPolygonTest(
        np.array(line, np.int32),
        ((x + int(tw_width/2),
          y + int(tw_height/2))),
        False)
    
    return line_crossing

In [ ]:
video = cv2.VideoCapture(r'.\input_data\2023_03_14_PTM.avi')

fullsize_first_frame = video.read()[1]

roi_start_x = 315
roi_end_x = 370
roi_start_y = 207
roi_end_y = 400
first_frame = fullsize_first_frame[roi_start_x:roi_end_x, roi_start_y:roi_end_y]

%matplotlib widget
plt.imshow(fullsize_first_frame)

In [ ]:
video = cv2.VideoCapture(r'.\input_data\2023_03_14_ptm.avi')

roi_start_x = 315
roi_end_x = 370
roi_start_y = 207
roi_end_y = 400

tw_start_point_x = 30
tw_start_point_y = 0
tw_start_width = 25
tw_start_height = 60
term_criteria= (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,1)

lines_list = [9,25,41,55,71,87,102,119,133,149,164,180]
line_labels = ['-6','-5','-4','-3','-2','-1','0','1','2','3','4','5']

while True:
    orig_frame = video.read()[1]
    frame = orig_frame[roi_start_x:roi_end_x, roi_start_y:roi_end_y]  
    enhanced_frame, threshold = get_binary_image(frame)
    track_window = cv2.meanShift(threshold,
                                 (tw_start_point_x, tw_start_point_y,
                                  tw_start_width, tw_start_height),
                                 term_criteria)[1]
    
    rows, cols, _bands = enhanced_frame.shape
    tw_start_point_x, tw_start_point_y, tw_width, tw_height = track_window
    tracking_window_geometry = display_tracking_window_axis_center(
         enhanced_frame,
         tw_start_point_x, tw_start_point_y,
         tw_width, tw_height,rows)
    bounding_box, horisontal_axis, object_center = tracking_window_geometry

    line_crossing_check = []
    for lines in lines_list:
        draw_lines(enhanced_frame,lines,tw_start_point_y,tw_height)
        crossing = check_line_crossing(lines,
                                       tw_start_point_x, tw_start_point_y,
                                       tw_width, tw_height)
        line_crossing_check.append(crossing)

    for line, next_line, scale_label in zip(line_crossing_check[1:11],
                                            line_crossing_check[2:11],
                                            line_labels[2:11]):
            if line == 1:
                initial_scale_time = datetime.now()
                initial_scale_timestamp = datetime.timestamp(initial_scale_time)
            if next_line == 1:
                next_scale_time = datetime.now()
                next_scale_timestamp = datetime.timestamp(next_scale_time)
                scale_crossing = next_scale_timestamp-initial_scale_timestamp
                scale_crossing = round(scale_crossing,3)
                
                crossing_index = line_crossing_check.index(next_line)
                crossing_index = scale_label
                print(f'Scale:{scale_label} Seconds:{scale_crossing}')                

    cv2.imshow('Frame', enhanced_frame)
    key = cv2.waitKey(100) #for framerate 20, normal playback is 100
    if key == 27:
        break
video.release()
cv2.destroyAllWindows()